In [1]:
%autosave 5

Autosaving every 5 seconds


In [2]:
import actr
import pickle
import random
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict,Counter
from itertools import groupby,combinations
import json
from IPython.display import Image 
import math
%matplotlib inline

ACT-R connection has been started.


### Existing Problems
    
    1. BIG PROBLEM: The items recalled in the first list are somehow coming from the second list as well.
    Ex: Original Lists 
    {0: {0: 'neutral537', 1: 'neutral871', 2: 'negative262', 3: 'neutral154', 4: 'positive552'}, 
    1: {0: 'neutral281', 1: 'neutral889', 2: 'negative842', 3: 'positive771', 4: 'neutral457'}, 
    2: {0: 'neutral388', 1: 'neutral429', 2: 'positive213', 3: 'negative659', 4: 'neutral848'}, 
    3: {0: 'neutral13', 1: 'neutral553', 2: 'positive179', 3: 'negative293', 4: 'neutral935'}})
    ------------------------------
    ------------------------------
    Recalled Lists 
    {0: ['neutral388', 'positive213', 'neutral457', 'negative842', 'negative659', 'neutral889', 'neutral848'], 
    1: ['neutral388', 'positive213', 'neutral457', 'negative842', 'negative659', 'neutral889', 'neutral848'], 
    2: ['neutral388', 'positive213', 'neutral457', 'negative842', 'negative659', 'neutral889', 'neutral848'], 
    3: ['neutral388', 'positive213', 'neutral457', 'negative842', 'negative659', 'neutral889', 'neutral848']})
    
    
    2. There is no recall many times even with 0 delay which should never be the case.
    3. The lists always contains 2 more neutral words (for primacy) was it the same in original experiments?

### Postman lag-CRP

    - Hyperparameters: list_amount, list_length, delay, recall_time, no_of_agents
    - list_amount and no_of_agents : 30 to keep it statistically significant
    - list_length and delay are varied 
    - recall time is 60 seconds as in Postman's original experiment

In [3]:
### Experiment part ###
def __init__(iteration):
    subject = ''

    current_list = ''

    list_amount = 1   # No of lists (100,200, 1000, 2000 AND 5000)

    list_length = 30   # No of words in a list

    rehearsal_time = 1  # No of seconds for which rehearsal happens and each word is shown

    delay = 0  #delay between rehearsal and recall

    recall_time = 60
    
    list_keys = list(range(list_amount))
    value = []
    recalled_words = dict.fromkeys(list_keys,value)
    
    rehearsed_words =  defaultdict(lambda: defaultdict(int))

    word_lists_dict = defaultdict(list)
    word_indices_dict = defaultdict(dict)
   
    # Ensure there are enough unique words to create the word lists
    word_dict = {"positive": ["positive" + str(i) for i in range(999)],
                 "negative": ["negative" + str(i) for i in range(999)],
                 "neutral": ["neutral" + str(i) for i in range(999)]}
   
      # Ensure there are enough unique words to create the word lists
#     word_dict = {"positive": ["positive" + str(i) for i in range(list_length)],
#                  "negative": ["negative" + str(i) for i in range(list_length)],
#                  "neutral": ["neutral" + str(i) for i in range(list_length)]}

    all_unique_words = set()
    
    extra_list_intrusions = 0
    
    if not os.path.exists(f'lag_crp_plots_postman/delay_{delay}'):
        os.mkdir(f'lag_crp_plots_postman/delay_{delay}')
        
#     if not os.path.exists(f'./lag_crp_plots/delay_{delay}/words_{list_length}'):
#         os.mkdir(f'./lag_crp_plots/delay_{delay}/words_{list_length}')
        
    filename = f'lag_crp_plots_postman/delay_{delay}/words_{list_length}.txt'
    
    results = {}



    # results['x'] = {'data': [], # will be appended later in the analytics function
    #                      'info': "Storing range(len(word_lists_dict[0])) here"}

    results['rehearse_frequency'] = {'data': None,# will be appended later in the analytics function

                                     'info': "Storing list(rehearse_frequency.values()) here"}

    results['recall_probability'] = {'data': None, # will be appended later in the analytics function
                                     'info': "Storing list(recall_probability.values()) here"}

    results['first_recall'] = {'data': None, # will be appended later in the analytics function
                               'info': "Storing list(first_probability.values()) here"}
    
    results['lag_crps'] = {'data': None, # will be appended later in the analytics function
                               'info': "Storing lag_crps for each agent here"}

    with open(filename, 'w') as outfile:
        json.dump(results, outfile)

    with open(filename) as json_file:
        results = json.load(json_file)
        #print(results)
    
    globals().update(locals())  ## Making everything public, worst code you can ever write!!

In [4]:
def add_words(i, list_length):
    '''
    Add the words to the word lists, ensures valence categories are balanced
    '''
    global word_lists_dict,word_indices_dict

    amnt_wanted = (list_length -2)/3   # Amount of each valence wanted, minus 2 neutrals controlling for primacy
    amt_positive, amt_negative, amt_neutral, count = 0, 0, 0, 0
    while len(word_lists_dict[i]) != list_length:
        count += 1
        #print(f"...................{count,word_lists_dict[i]}")
        if count >= 9999: # IF it takes too long to create a unique list at random, start over
            word_lists_dict[i] = []
            add_words(i, list_length)
        if len(word_lists_dict[i]) == 0: # Place two neutral words at the start to control for primacy effects
            word_to_add1 = word_dict["neutral"][random.randint(0, len(word_dict["neutral"])-1)]
            word_to_add2 = word_dict["neutral"][random.randint(0, len(word_dict["neutral"])-1)]
            if word_to_add1 not in word_lists_dict[i] and word_to_add2 not in word_lists_dict[i] and word_to_add1 != word_to_add2:
                word_lists_dict[i].append(word_to_add1)
                word_lists_dict[i].append(word_to_add2)
            else:
                continue # skip this loop iteration                   
        else: 
            random_valence = random.choice(["positive", "negative", "neutral"])
            word_to_add = word_dict[random_valence][random.randint(0, len(word_dict[random_valence])-1)]
            if word_to_add not in word_lists_dict[i] and word_lists_dict[i][-1] not in word_dict[random_valence] and \
               amt_positive <= amnt_wanted and amt_negative <= amnt_wanted and amt_neutral <= amnt_wanted:
                if random_valence == "positive" and amt_positive < amnt_wanted:
                    amt_positive += 1
                elif random_valence == "negative" and amt_negative < amnt_wanted:
                    amt_negative +=1
                elif random_valence == "neutral" and amt_neutral < amnt_wanted:
                    amt_neutral +=1
                else:
                    continue # skip this loop iteration
                word_lists_dict[i].append(word_to_add)
                
    #convert the list to dictionary 
    word_indices_dict[i] = dict(zip(list(range(list_length)),word_lists_dict[i]))
    all_unique_words.update(word_lists_dict[i])
    
def create_lists(list_amount=3, list_length=2):
    '''
    Create the wordlists used during the free recall tasks 
    '''  
    global word_lists_dict 
    print(f'Creating {list_amount} lists with {list_length} words.')
    for i in range(list_amount):
        print(f'List {i+1}/{list_amount} created!', end="\r")
        add_words(i, list_length)

    # Save the dictionary to a .pickle file, so we do not have to create the word lists everytime we run the model                    
    file = open(f"word_lists/word_lists_dict_{list_length}_{list_amount}.pickle","wb")
    pickle.dump(word_lists_dict, file)
    file.close()
    return word_lists_dict

# Check if the word lists already exist, else create new word lists
def check_and_create_lists():
    global word_lists_dict
    try:
        file = open(f"word_lists/word_lists_dict_{list_length}_{list_amount}.pickle","rb")
        #file = open(f"word_lists_dict_100_items_only.pickle","rb")
        word_lists_dict = pickle.load(file)
        for k,v in word_lists_dict.items():
            #print(k,v)
            word_indices_dict[k] = dict(zip(list(range(list_length)),v))
            all_unique_words.update(v)
        #word_indices_dict[i] = dict(zip(list(range(list_length)),word_lists_dict[i]))
        file.close()
        print("\nSuccesfully loaded the word lists!\n")
    except FileNotFoundError:
        print("\nCreating word lists!\n")
        #amount_to_create = list_amount                              
        word_lists_dict = create_lists(list_amount,list_length)

def display_word_lists():
    '''
    Display the word lists loaded/created
    '''
    for key, value in word_lists_dict.items():
        print(f'List {key}:\n {value}\n')
#     for key, value in word_indices_dict.items():
#         print(f'List {key}:\n {value}\n')

def close_exp_window():
    '''
    Close opened ACT-R window
    '''
    return actr.current_connection.evaluate_single("close-exp-window")

def prepare_for_recall(): 
    '''
    Disable rehearsing productions, and clearing buffer contents to start the recalling phase 
    '''
    disable_list = ["rehearse-first", "rehearse-second", "rehearse-third", "rehearse-fourth", 
                    "rehearse-it", "skip-first", "skip-second", "skip-third", "skip-fourth"]
    for prod in disable_list:
        actr.pdisable(prod)
    actr.run(1, False) 
    for buff in ["imaginal", "retrieval", "production"]:
        actr.clear_buffer(buff)  

def setup_dm(word_list):
    '''
    Add words to declarative memory, since it can be assumed the test subjects know the English language already
    '''
    #print("\n\n############################################# Inside setup_dm i.e. Declarative Memory")
     
    colour_conversion = {'pos': 'GREEN', 'neg': 'RED', 'neu': 'BLACK'}
    for idx, word in enumerate(word_list):
        valence = ''.join([char for char in word if not char.isdigit()])[:3]
        actr.add_dm(('item'+str(idx), 'isa', 'memory', 'word', "'"+word+"'", 'valence', colour_conversion[valence]))
#         if idx == 0:
#             print("\n Emaple of a chunk added in Declarative Memory is \n")
#             print('item'+str(idx), 'isa', 'memory', 'word', "'"+word+"'", 'valence', colour_conversion[valence],"\n")
        

def setup_experiment(human=True):
    '''
    Load the correct ACT-R model, and create a window to display the words
    '''
#     print("\n\n############################################# Inside setup_experiment")
#     print(f'\nSubject = {subject}\n')  

    loaded = None
    if subject == "controls":
        loaded = actr.load_act_r_model("C:/Users/brata/Desktop/free_recall_cognitive_models/fyrp/csm_free_recall_model_v1.lisp")
    elif subject == "depressed":
        loaded = actr.load_act_r_model("C:/Users/brata/Desktop/free_recall_cognitive_models/fyrp/csm_free_recall_model_depressed.lisp")

    #print("\n\n############################################# Inside setup_experiment")
    #print(f'\nLoaded Act-r model = {loaded}\n')  



    window = actr.open_exp_window("Free Recall Experiment", width=1024, height=768, visible=human) # 15inch resolution window
    actr.install_device(window) 
    return window    

def record_words_recalled(item):
    '''
    Register which words were recalled during the experiment for a specific wordlist and strip the numbers
    '''
    #print(f"in record_words_recalled, the item is {item}")
    valence = ''.join(char for char in item if not char.isdigit())
    item_idx = ''.join(char for char in item if char.isdigit())
    recalled_words[current_list].append((valence, item_idx))

def record_words_rehearsed(item):
    '''
    Register amount of rehearsals per word for each wordlist
    '''
    rehearsed_words[current_list][item] += 1

def create_lplot(idx, xlabel, ylabel, x, y, xticks_len, filename, ytick_range=None, show=False):
    '''
    Create line plot using matplotlib
    '''
    plt.figure(idx)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x, y)
    plt.xticks(np.arange(0, xticks_len, 1)) 
    plt.yticks(ytick_range)
    plt.savefig("images/"+subject+"_"+filename, bbox_inches='tight')
    if show:
        plt.show()    

        
def create_result_dict():
    '''
    Use a module-level function, instead of lambda function, to enable pickling it
    '''
    return defaultdict(int)


def modify_recalled_words_for_lag(recalled_words):
    """
    return joined values in recalled words dictionaries
    
    Ex: ('negative',781) ==> negative781
    """
    modified_recalled_words = defaultdict(list)
    for k,vals in recalled_words.items():
        #temp =[]
        for v in set(vals):
            #temp.append("".join(v))
            #print("temp",temp)
            modified_recalled_words[k].append("".join(v))
    #print("mod",modified_recalled_words)
    return modified_recalled_words

def calculate_lag(word_indices_dict, modified_recalled_words):
    """
    We are calculating the lag for each list and each recall.
    Extra_list_intrusions and repeated recalls are ignored in the calculations
    First verify that recall is happening after every list.
    
    """
    global extra_list_intrusions
    #print(word_indices_dict,recalled_words)
    #for i in range(list_amount):
        
    #check ith list recalled words and indices
    lags = defaultdict(list)
    for k,vals in modified_recalled_words.items():
        
        #print("vals in this k", k, vals, word_indices_dict[k])
        for j in range(1,len(vals)):
            #print("testing recall",vals[j],word_indices_dict[k])
            if vals[j] in word_indices_dict[k].values() and vals[j-1] in word_indices_dict[k].values():
                #print("INsiiiide")
                current_index = list(word_indices_dict[k].keys())[list(word_indices_dict[k].values()).index(vals[j])]
                prev_index = list(word_indices_dict[k].keys())[list(word_indices_dict[k].values()).index(vals[j-1])]
                #print("indices" ,current_index,prev_index)
                lags[k].append(current_index - prev_index)
            if vals[j] not in word_indices_dict[k].values() and vals[j] in all_unique_words:
                extra_list_intrusions += 1
                continue
    #print("lagsssss",lags)           
    return lags

In [5]:
## Creating different pickle files to store results from multiple hyper-parameter values.
def analysis(wlist_amount, show_plots=False):
    '''
    Review results of the recall experiment
    '''
    global results
    result_dict = defaultdict(create_result_dict) # instead of defaultdict(lambda: defaultdict(int))
    first_recall = defaultdict(int)
    recall_probability = defaultdict(int)
    rehearse_frequency = defaultdict(int)
    transitions_amnt = 0
    thought_train_len = []

    for key, val in recalled_words.items():
        thought_train_len.extend([(k, sum(1 for _ in count)) for k, count in groupby([val[0] for val in val[2:]])])
        for idx, (retrieved_word, item_num) in enumerate(val[2:]):
            if idx != 0:
                if retrieved_word != val[2:][idx-1][0]:
                    transitions_amnt += 1/wlist_amount # average over word lists

    print(f'Avg. Amount of recall transitions = {int(transitions_amnt)}')
    neg_thought_train_len = 0
    neg_divider = 0.0001
    for x in thought_train_len:
        if x[0] == 'negative':
            neg_divider += 1
            neg_thought_train_len += x[1]
    print(f'Avg. Negative Thought train length = {round(neg_thought_train_len/neg_divider, 3)}')            

    for list_num, wlist in word_lists_dict.items():
        if list_num < wlist_amount:
            for key, val in recalled_words.items():
                if key==list_num:
                    first_recall[wlist.index(''.join(val[0]))] += 1   
                    for idx, word in enumerate(wlist):
                        first_recall[idx] += 0
                        if ((''.join(char for char in word if not char.isdigit()), 
                             ''.join(char for char in word if char.isdigit()))) in val:
                            recall_probability[idx] += 1
                        else:
                            recall_probability[idx] += 0                            
                for retrieved_word, item_num in val[2:4]:
                    result_dict["pstart"][retrieved_word] += 1  
                for retrieved_word, item_num in val[4:-2]:
                    result_dict["pstay"][retrieved_word] += 1
                for retrieved_word, item_num in val[-2:]:
                    result_dict["pstop"][retrieved_word] += 1                                                        
            for key, val in rehearsed_words.items():
                if key==list_num:
                    for idx, word in enumerate(wlist):
                        rehearse_frequency[idx] += rehearsed_words[key][word]
    
    for key, val in first_recall.items():
        first_recall[key] = val/wlist_amount

    for key, val in recall_probability.items():
        recall_probability[key] = val/wlist_amount

    for key, val in rehearse_frequency.items():
        rehearse_frequency[key] = val/wlist_amount      
        
    

    xticks_len = len(word_lists_dict[0])
    
    
    #results['x']['data'].append(range(len(word_lists_dict[0])))
    #results['xticks_len']['data'].append(len(word_lists_dict[0]) )
    results['rehearse_frequency']['data'] = list(rehearse_frequency.values())
    results['recall_probability']['data'] = list(recall_probability.values())
    results['first_recall']['data'] = list(first_recall.values()) 
    
    with open(filename, 'w') as outfile:
        json.dump(results, outfile)
        
    create_lplot(0, 'Serial input position', 'Rehearse Frequency', range(len(word_lists_dict[0])), list(rehearse_frequency.values()), 
                xticks_len, f'rehearse_frequency_{list_length}_{list_amount}_{rehearsal_time}_{recall_time}_{delay}.png', None, show_plots)

    create_lplot(1, 'Serial input position', 'Starting Recall', range(len(word_lists_dict[0])), list(first_recall.values()), 
                xticks_len, f'starting_recall_{list_length}_{list_amount}_{rehearsal_time}_{recall_time}_{delay}.png', np.arange(0, .5, .1), show_plots)                

    create_lplot(2, 'Serial input position', 'Recall Probability', range(len(word_lists_dict[0])), list(recall_probability.values()), 
                xticks_len, f'recall_probability_{list_length}_{list_amount}_{rehearsal_time}_{recall_time}_{delay}.png', np.arange(0, 1, .1), show_plots)   
    
#     create_lplot(3, 'Serial input position', 'Accuracy', range(len(word_lists_dict[0])), list(recall_accuracy.values()), 
#                 xticks_len, 'recall_accuracy.png', np.arange(0, 1, .1), show_plots) 

    file = open("results_"+subject+".pickle","wb")
    pickle.dump(result_dict, file)
    file.close()

    return result_dict

In [6]:
def do_experiment(subj="depressed", human=False, wlist_amount=20):
    '''
    Run the experiment
    '''
    check_and_create_lists()
    global subject,word_lists_dict
    subject = subj
    assert wlist_amount <= len(word_lists_dict), "Chosen too many lists, choose less or create more word lists using function: create_lists()"
    
#     print("###################################################\n")
#     print("The original word list \n")
#     print(display_word_lists())
   
    
#     print("\n###################################################\n")
#     print("Experiment started, Trying to understand the flow\n")
  
    for idx, (key, value) in enumerate(word_lists_dict.items()):
        actr.reset()
        window = setup_experiment(human)
        global current_list
        current_list = idx # keep track for which list words are recalled
        setup_dm(value)   
        actr.add_command("retrieved-word", record_words_recalled,"Retrieves recalled words.")
        actr.add_command("rehearsed-word", record_words_rehearsed,"Retrieves rehearsed words.")
#         print("\n##################  Model started rehearsal ")
        for word in value:
            if "neutral" in word:
                color = "black"
            elif "positive" in word:
                color = "green"
            else:
                color = "red"
            actr.add_text_to_exp_window(window, word, x=475-len(word) , y=374, color=color, font_size=20) # change later 
            actr.run(rehearsal_time, human) # True when choosing Human, False when choosing differently
            actr.clear_exp_window(window)
            actr.run(delay, human)  # 500-ms blank screen                        
        prepare_for_recall()       
        actr.remove_command("rehearsed-word")
#         print("\n##################  Model finished rehearsal, list of rehearsed words is ")
#         print(f'{rehearsed_words}\n')
#         print("\n##################  Model started recall ")
        actr.goal_focus("startrecall") # set goal to start recalling
        actr.run(recall_time, human)  
        actr.remove_command("retrieved-word")

        print(f'Experiment {idx+1}/{wlist_amount} completed!', end="\r")
        #print("\n##################  Model finished recall, list of recalled words is ")
        #print(f'{recalled_words}\n')
        if idx == wlist_amount-1: # run for a chosen amount of word lists
            break
    close_exp_window() # close window at end of experiment     

    avg_recalled, avg_recalled_unique = 0, 0
    for key, val in recalled_words.items():
        avg_recalled += len(val)
        avg_recalled_unique += len(set(val))
    #    print(f'\nList {key} (length={len(val)}, unique={len(set(val))})')
    print(f'Avg. Amount of words recalled = {avg_recalled//wlist_amount}')
    print(f'Avg. Amount of unique words recalled = {avg_recalled_unique//wlist_amount}')
    
    #print(f'recalled words {recalled_words}')
    
#     result = analysis(list_amount, False)        

#     for key, val in result.items():
#         print(f'{key} = {dict(val)}')
#     print()
 

#     print("\n\n#############################################")
#     print(f'\n[{subj}] Results!\n')  

In [7]:
def count_possible_lags(list_length):
    """
    Given the list length we will count all the possible lags by calculating the pairs for each lag
    """
    possible_lags = list(combinations(list(range(list_length)),2))
    #print(possible_lags,len(possible_lags))
    possible_lags_count = defaultdict(int)
#     possible_lags_count[0] = 0
#     possible_lags_count[1] = list_length-1
#     possible_lags_count[-1] = list_length-1
    
    for lag in possible_lags:
        possible_lags_count[lag[0]-lag[1]] += 1
        possible_lags_count[lag[1]-lag[0]] += 1
        #possible_lags_count[-lag] += math.floor(list_length/lag)
    return possible_lags_count

In [8]:
def splitSerToArr(ser):
    return [ser.index, ser.values]

In [9]:
def plot(cummulative_lags):
    fig, ax = plt.subplots(1,1)
    x_vals = list(cummulative_lags.keys())
    y_vals = list(cummulative_lags.values())
    x_vals_pos = []
    y_vals_pos = []
    x_vals_neg = []
    y_vals_neg = []
    # Dividing the lists into two parts negative and positive to achieve the curves style in Kahana
    for i in range(len(x_vals)):
        if x_vals[i]<0:
            x_vals_neg.append(x_vals[i])
            if y_vals[i]!=0:
                y_vals_neg.append(y_vals[i])
            else:
                y_vals_neg.append(None)
        elif x_vals[i]>0:
            x_vals_pos.append(x_vals[i])
            if y_vals[i]!=0:
                y_vals_pos.append(y_vals[i])
            else:
                y_vals_pos.append(None)
        else:
            continue
    
#labels = ['Geeks1', 'Geeks2', 'Geeks3', 'Geeks4'] 
    #Converting the arrays in Series(pandas) to convert 0's to NaNs so that they are not plotted
    s_pos = pd.Series(y_vals_pos, index=(x_vals_pos))
    s_neg = pd.Series(y_vals_neg, index=(x_vals_neg))
    #plt.plot( *splitSerToArr(s_pos.dropna()), linestyle='-', marker='o')
    plt.ylabel('Conditional Response Probability')
    plt.xlabel('Lags values (ranging from -list_length+1 to list_length-1)')
    #plt.grid(True)
    plt.ylim((0,1))
    #plt.xticks(list(range(-list_length+1,list_length,1)))
    #plt.margins(0) 
    # Tweak spacing to prevent clipping of tick-labels 
    plt.subplots_adjust(bottom = 0.15)
    ax.plot(*splitSerToArr(s_pos.dropna()), '-o')
    ax.plot(*splitSerToArr(s_neg.dropna()), '-o')
    ax.set_xticks(list(range(-list_length+1,list_length,1)))
    plt.savefig(f"lag_crp_plots_postman/delay_{delay}/single_experiment_lag_crp_words_{list_length}_lists_{list_amount}.png")
    plt.show()

In [10]:
# crp = {0: {-7: 0.25, -1: 0.1}, 1: {-9: 0.5}}
# dict_keys = list(range(-list_length+1,list_length))
# dict_val = 0
# cummulative_lags = dict.fromkeys(dict_keys,dict_val)

# for value in range(-list_length+1,list_length): #key corresponds to number of lists and vals are the lags inside each list
#     value_count = 0 # Number of times this lag occured in all lists
#     for k,v in crp.items(): #k represents different lags that occur
#         #print("k, ",k,crp[k].keys())
#         if value in crp[k].keys():
#             value_count+=1
#             cummulative_lags[value] += crp[k][value]
#     if value_count!=0:
#         cummulative_lags[value] /= value_count
# print("Cummulative lags", cummulative_lags)
# #Cummulative lags defaultdict(<class 'int'>, {-10: 0, -9: 0.5, -8: 0, -7: 0.0, -6: 0, -5: 0, -4: 0, -3: 0, -2: 0, -1: 0.0, 0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0})

In [11]:
def calculate_lag_crp(lags):
    """
    Calculates average lags across all the lists 
    Response probability is calculated by the number_of_transtions of the lag/total transitions possible of the lag
    Note: Total Transitions does not count the extra_list_intrusions or repeated recalls
    """
    # Gets dictionary containing total possible lags in the list of length list_length.
    possible_lags_count = count_possible_lags(list_length)
    #print("received",lags)
    crp = defaultdict(dict)
    for key,vals in lags.items():
        #lags that occured in the current list
        #count the different lags in the lags list
        counts = Counter(vals)
        for k,v in counts.items():
            crp[key][k] = v/possible_lags_count[k]
    #print("CRP ", crp)
    dict_keys = list(range(-list_length+1,list_length))
    dict_val = 0
    cummulative_lags = dict.fromkeys(dict_keys,dict_val)
    for value in range(-list_length+1,list_length): #key corresponds to number of lists and vals are the lags inside each list
        value_count = 0 # Number of times this lag occured in all lists
        for k,v in crp.items(): #k represents different lags that occur
            if value in crp[k].keys():
                value_count+=1
                cummulative_lags[value] += crp[k][value]
        if value_count!=0:
            cummulative_lags[value] /= value_count
    #print("Cummulative lags", cummulative_lags)
    #print("crps", crp)
    #plot(cummulative_lags)
    return cummulative_lags

In [12]:
num_agents = 30
cummulative_lags_all_agents = []
for agent in range(num_agents):
    print("------------------------------")
    print(f"Started for agent_{agent}")
    __init__(agent)
    do_experiment('controls',False,list_amount)
#     print("indices",word_indices_dict)
#     print("------------------------------")
#     print("------------------------------")
    modified_recalled_words = modify_recalled_words_for_lag(recalled_words)
#     print("recalled_words",modified_recalled_words)
#     print("------------------------------")
#     print("------------------------------")
    #print(all_unique_words)
    lags = calculate_lag(word_indices_dict, modified_recalled_words)
    #print(word_indices_dict)
    #print(recalled_words)
    #print(lags)
    cummulative_lags_all_agents.append(calculate_lag_crp(lags))

------------------------------
Started for agent_0


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'lag_crp_plots_postman/delay_0'

In [ ]:
results['lag_crps']['data'] = cummulative_lags_all_agents
with open(filename, 'w') as outfile:
    json.dump(results, outfile)

In [ ]:
# def test_calculate_lag(word_indices_dict, modified_recalled_words):
#     """
#     We are calculating the lag for each list and each recall.
#     Extra_list_intrusions and repeated recalls are ignored in the calculations
#     First verify that recall is happening after every list.
    
#     """
#     global extra_list_intrusions
#     #print(word_indices_dict,recalled_words)
#     #for i in range(list_amount):
        
#     #check ith list recalled words and indices
#     lags = defaultdict(list)
#     for k,vals in modified_recalled_words.items():
        
#         #print("vals in this k", k, vals, word_indices_dict[k])
#         for j in range(1,len(vals)):
#             print("testing recall",vals[j],vals[j-1])
#             if vals[j] in word_indices_dict[k].values() and vals[j-1] in word_indices_dict[k].values():
#                 #print("INsiiiide")
#                 current_index = list(word_indices_dict[k].keys())[list(word_indices_dict[k].values()).index(vals[j])]
#                 prev_index = list(word_indices_dict[k].keys())[list(word_indices_dict[k].values()).index(vals[j-1])]
#                 print("indices" ,current_index,prev_index)
#                 lags[k].append(current_index - prev_index)
#             if vals[j] not in word_indices_dict[k].values() and vals[j] in all_unique_words:
#                 extra_list_intrusions += 1
#                 continue
#     #print("lagsssss",lags)           
#     return lags

In [ ]:
# word_indices_dict = {0: {0: 'neutral732', 1: 'neutral886', 2: 'negative209', 3: 'positive991', 4: 'neutral146', 5: 'positive53', 
#                         6: 'negative459', 7: 'positive940', 8: 'neutral547', 9: 'negative582', 10: 'neutral142'}, 
#                     1: {0: 'neutral346', 1: 'neutral464', 2: 'negative683', 3: 'neutral982', 4: 'negative661', 5: 'neutral712',
#                         6: 'positive289', 7: 'negative560', 8: 'positive636', 9: 'neutral933', 10: 'positive634'}}

# m_recalled_words = {0: [('positive', '53'), ('positive', '53'), ('negative', '582'), ('neutral', '346'), ('positive', '636'), ('neutral', '712')], 
#                   1: [('positive', '53'), ('positive', '53'), ('negative', '582'), ('neutral', '346'), ('positive', '636'), ('neutral', '712')]}

# mod_recalled_words = modify_recalled_words_for_lag(m_recalled_words)

# print("recalled words ",mod_recalled_words)
# lags = test_calculate_lag(word_indices_dict, mod_recalled_words)